# Energy Market Analysis - Task 1


#### API used: [ US Energy Information Administration ](https://www.eia.gov/opendata)


**Objective:** As society increasingly moves towards cleaner energy generation with renewable energy sources and we see a change in attitude towards fossil fuels, I would like to investigate the correlation between Gas prices and Electricity prices over recent years and the effect this change in view has had on said prices. I will also evaluate the U.S. consumption trends of both energy sources, and how it has changed over recent years and investigate the usage of `Natural Gas to generate `Electricity`.

**Data Collected:** 
- Starting from `January 2000` up until as recent as the available data from the API `(December 2022)`, I retrieve historic monthly data of the average Gas and Electricity prices for the ultimate customer in the US. This includes prices for Residential, Industrial, and Commercial as per EIA API.
- Data is then also collected regarding, the consumption of renewables, natural gas and natural gas consumer to generate electricity.
 
 
#### Idea Inspiration for Chosen Topic:  
The below articles assited in my brainstorming for the chosen topic.
- [ Medium Article: Why Does Gas Set the Price of Renewable Energy? ](https://medium.com/climate-conscious/why-does-gas-set-the-price-of-renewable-energy-6780d7d856a9)
- [Why does the price of gas drive electricity prices, including renewables?](https://www.goodenergy.co.uk/why-does-the-price-of-gas-drive-electricity-prices-including-renewables/)
- [Renewables and The Electric Grid](https://medium.com/@dpickut2/renewables-and-the-electric-grid-27ab4b693969)



In [1]:
import json, requests, urllib, csv
from pathlib import Path
import pandas as pd

In [2]:
# Prefix for API & URL
url = "https://api.eia.gov/v2/total-energy/data?api_key=<INSERT_YOUR_API_KEY>"

### Method Outline
There was no Total Natural Gas Average Price data point from the API as I had used for Electricity price data, I found that some of the data points, e.g. Gas Price, for the transport sector (NGACBUS), contained mainly incomplete data when implementing test requests on the website API.

In order to work out an Average National Natural Gas Price for each of the historic months, I obtained the sectors in which there were sufficient records: Residential, Commercial and Industrial. Following this, I used the mean of these three sectors to calculate a 'total' value for each month in the dataset.

### Gas Prices Data

In [3]:
# Load Gas prices
gas_headers = {'X-Params': json.dumps({
    "frequency": "monthly",
    "data": [
        "value"
    ],
    "facets": { # Facets to calculate the total Gas prices
        "msn": [
            "NGCCUUS", # Gas Price Commercial
            "NGINUUS", # Gas Price Industrial
            "NGRCUUS"  # Gas Price residential
        ]
    },
    "start": "2000-01",
    "end": "2023-02",
    "sort": [
        {
            "column": "period",
            "direction": "asc"
        }
    ],
    "offset": 0,
    "length": 5000 # Max value the data set allows.
})}

gResponse = requests.get(url, headers=gas_headers)

if gResponse.status_code == 200:
    data = gResponse.json()    
else:
    print(f"Error {gResponse.status_code}: {gResponse.text}")


# Convert JSON response to a dictionary
gResponse_dict = json.loads(gResponse.text)

# Store data from response
gas_data = gResponse_dict['response']['data']

Once the data is taken from the API for the Gas Prices I then add this to a panda DataFrame & group the 3 different sectors based on the `period` key date value. I store the mean value of the three sectors for each month to give a total price for each month.

In [4]:
# Create a DataFrame from the response of gas prices
gdf = pd.DataFrame(gas_data)

# Calculate average monthly gas price using the three industries: (Residential, Industrial & Commeercial)
totaldf = gdf.groupby('period')['value'].mean().round(2)
gdf = gdf.pivot_table(values='value', index='period', columns='msn')
gdf.head()

msn,NGCCUUS,NGINUUS,NGRCUUS
period,,,
2000-01,5.77,3.41,6.37
2000-02,5.95,3.68,6.54
2000-03,5.78,3.54,6.91
2000-04,6.03,3.59,7.19
2000-05,5.97,3.67,8.26


In [5]:
# Check total gas prices DF
totaldf.head()

period
2000-01    5.18
2000-02    5.39
2000-03    5.41
2000-04    5.60
2000-05    5.97
Name: value, dtype: float64

In [6]:
# Concatenate the Values to obtain a full Gas Prices List
gdf['gas_total'] = totaldf
gdf

msn,NGCCUUS,NGINUUS,NGRCUUS,gas_total
period,,,,
2000-01,5.77,3.41,6.37,5.18
2000-02,5.95,3.68,6.54,5.39
2000-03,5.78,3.54,6.91,5.41
2000-04,6.03,3.59,7.19,5.60
2000-05,5.97,3.67,8.26,5.97
...,...,...,...,...
2022-08,14.24,9.76,25.52,16.51
2022-09,14.58,9.95,24.63,16.39
2022-10,12.84,7.38,18.72,12.98


### Electricity & Consumption Data
The GET request below provides the rest of the dataset: Total Electricity, Natural Gas Consumption for Electricity Generation, Natural Gas Consumption, Renewable Energy Production and Renewable Energy Consumption.

The data obtained is then merged with the previous pandas data frame I created of Average Gas prices.

In [7]:
# Extract Total Electricity Prices for 5 years
# API Description: Average Price of Electricity to Ultimate Customers, Total in Cents per Kilowatthour, Including Taxes

electricity_headers = {'X-Params': json.dumps({
    "frequency": "monthly",
    "data": [
        "value"
    ],
    "facets": { # Facets for API GET request
        "msn": [
            "REPRBUS", # Total Renew energy Production
            "RETCBUS", # Total Renewable energy consumption
            "NGEPPUS", # Natural Gas Consumption for Elec
            "NGTCPUS", #Total Natural Gas Consumption
            "ESTCUUS", #Total Elec Price
            "ESCMUUS", #Commercial Elec Price
            "ESICUUS", #Idustrial Elec Price
            "ESRCUUS", #Residential Elec Price
        ]
    },
    "start": "2000-01",
    "end": "2023-02",
    "sort": [
        {
            "column": "period",
            "direction": "asc"
        }
    ],
    "offset": 0,
    "length": 5000 # Max value the api allows.
})}

eResponse = requests.get(url, headers=electricity_headers)

if eResponse.status_code == 200:
    elect_data = eResponse.json()
    
else:
    print(f"Error {eResponse.status_code}: {eResponse.text}")

# Convert response to a dictionary
eResponse_dict = json.loads(eResponse.text)

# store data from response
elect_data = eResponse_dict['response']['data']

Create a dataframe from the API response for the given dates, create a pivot table to align the value data with each month, `period`, being the index.

In [8]:
# Create a DataFrame from response
edf = pd.DataFrame(elect_data)
edf = edf.pivot_table(values='value', index='period', columns='msn')

In [9]:
# Merge the dataframes on the period column, add the total gas prices column.
df = pd.merge(edf, gdf, on='period')

# reindex the columns
new_cols = ['ESRCUUS', 'ESCMUUS', 'ESICUUS', 'ESTCUUS', 'NGRCUUS','NGCCUUS', 'NGINUUS' , 'gas_total', 'NGEPPUS', 'NGTCPUS', 'REPRBUS', 'RETCBUS']
df = df.reindex(columns=new_cols)

df

msn,ESRCUUS,ESCMUUS,ESICUUS,ESTCUUS,NGRCUUS,NGCCUUS,NGINUUS,gas_total,NGEPPUS,NGTCPUS,REPRBUS,RETCBUS
period,,,,,,,,,,,,
2000-01,7.66,6.93,4.31,6.40,6.37,5.77,3.41,5.18,369.155,2532.443,505.154,505.528
2000-02,7.71,6.96,4.32,6.39,6.54,5.95,3.68,5.39,327.413,2340.654,499.963,498.999
2000-03,8.09,7.03,4.31,6.44,6.91,5.78,3.54,5.41,395.931,2085.459,558.010,558.482
2000-04,8.15,7.05,4.32,6.43,7.19,6.03,3.59,5.60,406.375,1827.336,568.530,567.156
2000-05,8.34,7.25,4.51,6.64,8.26,5.97,3.67,5.97,566.887,1736.144,558.730,558.834
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08,15.93,13.53,9.51,13.58,25.52,14.24,9.76,16.51,1398.719,2559.494,1044.026,1036.589
2022-09,16.31,13.45,9.22,13.49,24.63,14.58,9.95,16.39,1143.581,2288.491,978.647,960.694
2022-10,16.01,13.05,8.61,12.79,18.72,12.84,7.38,12.98,972.617,2366.092,1019.209,1006.859


Save the dataset obtained from the US Energy API in CSV file format to use in the second task.

In [10]:
# Write DF to output file
output_file = 'output.csv'
df.to_csv(output_file)